In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import huggingface_hub
from tqdm.auto import tqdm

In [2]:
# Hugging Face 로그인
huggingface_hub.login("")

In [3]:
# 모델 저장 위치 확인
print("모델 저장 위치:", huggingface_hub.constants.HF_HOME)

# 단일 GPU 사용을 강제
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"현재 사용 중인 장치: {device}")

모델 저장 위치: /home/sliver/.cache/huggingface
현재 사용 중인 장치: cuda:0


In [4]:
# 모델 및 토크나이저 로드
model_name = "meta-llama/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [5]:
# 모델 초기화 (메모리 사용량 감소를 위해 float16 타입 사용)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16
).to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Task 1

In [6]:
# 모델 평가 모드 설정
model.eval()

# 추론 실행
prompt = "Which is a fruit? (a) apple, (b) car, (c) dog"
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# 모델 추론 실행
with torch.no_grad():
    outputs = model.generate(
        **inputs,                             # 토크나이저로 변환된 입력 텍스트 (토큰화된 프롬프트)
        pad_token_id=tokenizer.eos_token_id,  # 패딩 토큰 ID (주로 EOS 토큰 ID 사용)
        max_new_tokens=512,                   # 최대 몇 개의 새 토큰을 생성할지 설정 (출력 길이 제한)
        do_sample=True,                       # True로 설정 시 확률 기반 샘플링 방식 사용 (False면 greedy decoding)
        temperature=0.7,                      # 출력 분포의 온도 조절 (낮을수록 확정적/보수적, 높을수록 무작위/창의적)
        top_k=10                              # 다음 토큰 예측 시 확률 상위 k개 토큰만 고려 (샘플링 범위 제한)
    )

# 응답 디코딩
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Which is a fruit? (a) apple, (b) car, (c) dog, (d) house

## Step 1: Identify the characteristics of a fruit.
A fruit is typically the edible part of a plant that grows from a flower and contains seeds. Examples include apples, bananas, and oranges.

## Step 2: Analyze each option to determine if it is a fruit.
- (a) Apple: This is a type of fruit.
- (b) Car: This is a vehicle, not a fruit.
- (c) Dog: This is an animal, not a fruit.
- (d) House: This is a structure, not a fruit.

## Step 3: Select the option that matches the definition of a fruit.
From the analysis, the only option that matches the definition of a fruit is the apple.

The final answer is: $\boxed{(a)}$


### Task 2

In [7]:
# 질문 데이터 10개 (출처 : tau/commonsense_qa 중 일부)
questions = [
    # 문제 1: 잠정 합의에 도달한 후 어떤 기분이 들까요? (정답: C - 만족감)
    # 보기: A - 불확실성, B - 긴장, C - 만족, D - 타협, E - 완료됨
    {"question": "What is someone likely to feel after reaching a tentative agreement?", 
     "choices": {"A": "uncertainty", "B": "relax", "C": "satisfaction", "D": "compromises", "E": "finished"},
     "answer": "C"},
    
    # 문제 2: 수년간의 노력 끝에 아이디어가 현실화되는 것을 보고 만족스러웠습니다. (정답: E - 축하하다)
    # 보기: A - 웃음, B - 불안, C - 빌드, D - 흥분, E - 축하하다
    {"question": "It was satisfying seeing idea become reality, after years of work he finally could what?", 
     "choices": {"A": "laugh", "B": "anxiety", "C": "build", "D": "excitement", "E": "celebrate"},
     "answer": "E"},
    
    # 문제 3: 아이디어가 현실화되는 것을 본 후 미래의 아이디어에 대한 느낌은 어떨까요? (정답: C - 영감)
    # 보기: A - 성취감, B - 가상 생활, C - 영감, D - 돈 벌기, E - 매우 좋다
    {"question": "What is likely to be the feeling for future ideas after seeing idea become reality?", 
     "choices": {"A": "fulfilling", "B": "virtual living", "C": "inspiration", "D": "make money", "E": "very nice"},
     "answer": "C"},
    
    # 문제 4: 크리스마스 선물 구매를 방해하는 요소는 무엇인가요? (정답: B - 돈 부족)
    # 보기: A - 행복, B - 돈 부족, C - 스트레스, D - 안도, E - 상쾌함
    {"question": "What could prevent someone from buying chistmas presents?", 
     "choices": {"A": "happiness", "B": "lack of money", "C": "stress", "D": "relief", "E": "exhiliration"},
     "answer": "B"},
    
    # 문제 5: 많은 사람들에게 당황스러운 상황을 말하면 어떻게 될까요? (정답: B - 정서적 고통)
    # 보기: A - 악명, B - 정서적 고통, C - 폭동, D - 나쁜 꿈, E - 짜증
    {"question": "What will telling many people about an embarrassing situation lead to?", 
     "choices": {"A": "notoriety", "B": "emotional distress", "C": "riot", "D": "bad dreams", "E": "annoyance"},
     "answer": "B"},
    
    # 문제 6: 새 조직에 대해 많은 사람들에게 알리는 목표는 무엇일까요? (정답: B - 배우다)
    # 보기: A - 성취하다, B - 배우다, C - 돈, D - 악명, E - 종교
    {"question": "What is likely to be the goal of telling many people about a new organization?", 
     "choices": {"A": "accomplish", "B": "learn about", "C": "money", "D": "notoriety", "E": "religion"},
     "answer": "B"},
    
    # 문제 7: 양측이 잠정적 합의에 도달할 수 있었다는 것은 어떤 성과인가? (정답: A - 성취)
    # 보기: A - 성취하다, B - 진전을 이루다, C - 논쟁을 멈추다, D - 사업하다, E - 콘서트
    {"question": "The fact the two sides were able to even reach tentative agreement was a feat to what?", 
     "choices": {"A": "accomplish", "B": "make progress", "C": "stop arguing", "D": "do business", "E": "concert"},
     "answer": "A"},
    
    # 문제 8: 샘은 크리스마스 선물 사는 것을 좋아했어요. 다른 사람에게 주는 것은 그에게 무엇을 느끼게 했나요? (정답: C - 행복)
    # 보기: A - 돈 부족, B - 즐거움, C - 행복, D - 사랑, E - 안도
    {"question": "Sam loved buying Christmas presents. Giving to others made him feel what?", 
     "choices": {"A": "lack of money", "B": "pleasure", "C": "happiness", "D": "love", "E": "relief"},
     "answer": "C"},
    
    # 문제 9: 잠정적 합의에 도달한 결과는 무엇인가요? (정답: B - 합의)
    # 보기: A - 소통하다, B - 합의, C - 악수, D - 축하, E - 헤어지다
    {"question": "What is the result of reaching a tentative agreement?", 
     "choices": {"A": "communicate with", "B": "settlement", "C": "shake hands", "D": "celebration", "E": "part ways"},
     "answer": "B"},
    
    # 문제 10: 채식주의자에 대해 많은 사람들에게 계속 말하면 어떤 위험이 있나요? (정답: A - 성가심)
    # 보기: A - 짜증, B - 성취, C - 취하다, D - 감정적 고통, E - 악명
    {"question": "If you keep telling many people about being a vegetarian what do you risk becoming?", 
     "choices": {"A": "annoyance", "B": "accomplish", "C": "intoxicated", "D": "emotional distress", "E": "notoriety"},
     "answer": "A"}
]

In [8]:
# 선택지를 "(a) option1, (b) option2" 형식으로 변환하는 함수
def format_choices(choices_dict):
    choices_text = []
    for label, text in choices_dict.items():
        choices_text.append(f"({label.lower()}) {text}")
    return ", ".join(choices_text)

In [9]:
# 모델 평가 모드 설정
model.eval()

# 모델 응답 결과를 저장할 리스트
results = []

# 각 질문에 대해 모델 응답 생성
for i, q in enumerate(tqdm(questions, desc="질문 처리 중")):
    # 원래 질문과 선택지를 합쳐서 프롬프트 생성
    formatted_choices = format_choices(q["choices"])
    prompt = f"{q['question']} {formatted_choices}"
    print(f"[질문 {i+1}] {prompt}")
    
    # 모델 입력 생성 및 장치 이동
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    # 모델 추론 실행
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            pad_token_id=tokenizer.eos_token_id,
            max_new_tokens=512,
            do_sample=True,
            temperature=0.7,
            top_k=10
        )
    
    # 응답 디코딩
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # 정답 확인
    correct_answer = q["answer"]
    correct_text = q["choices"][correct_answer]
    
    print(f"[응답 {i+1}]: \n {response}")
    print(f"[정답 {i+1}]: {correct_answer} - {correct_text}\n")
    print("================================================================================================================")
    # 결과 저장
    results.append({
        "question": prompt,
        "response": response,
        "correct_answer": f"{correct_answer} - {correct_text}"
    })

질문 처리 중:   0%|          | 0/10 [00:00<?, ?it/s]

[질문 1] What is someone likely to feel after reaching a tentative agreement? (a) uncertainty, (b) relax, (c) satisfaction, (d) compromises, (e) finished
[응답 1]: 
 What is someone likely to feel after reaching a tentative agreement? (a) uncertainty, (b) relax, (c) satisfaction, (d) compromises, (e) finished.
After reaching a tentative agreement, someone is likely to feel a sense of satisfaction, as they have managed to find a middle ground that meets both parties' needs. This is a common emotional response in situations where negotiations have been challenging and require compromise.
The best answer is (c).
[정답 1]: C - satisfaction

[질문 2] It was satisfying seeing idea become reality, after years of work he finally could what? (a) laugh, (b) anxiety, (c) build, (d) excitement, (e) celebrate
[응답 2]: 
 It was satisfying seeing idea become reality, after years of work he finally could what? (a) laugh, (b) anxiety, (c) build, (d) excitement, (e) celebrate
Answer: (e) celebrate
Explanation: T